In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LeakyReLU, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.metrics import classification_report, confusion_matrix
import keras.metrics as metrics
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools
import time
from keras.regularizers import l2
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:

# Ensure inline plotting
%matplotlib inline

# Set dataset paths
train_path = r'C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\sample images\5 classes\All layouts\train'
test_path = r'C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\sample images\5 classes\All layouts\test'

In [ ]:
# Prepare image data generators with augmentation for the training set
train_datagen = ImageDataGenerator(featurewise_center=False)

# Keep the test set generator simple (no augmentation)
test_datagen = ImageDataGenerator(featurewise_center=False)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    color_mode="rgb",
    classes=['action_1', 'action_2', 'action_3', 'action_4', 'action_5'],
    class_mode="categorical",
    batch_size=256,
    interpolation="nearest"
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    color_mode="rgb",
    classes=['action_1', 'action_2', 'action_3', 'action_4', 'action_5'],
    class_mode="categorical",
    batch_size=256,
    interpolation="nearest"
)


In [ ]:
# Initialize the CNN
Classifier = Sequential()

# Add Input layer
Classifier.add(Input(shape=(64, 64, 3)))

# First Convolutional Layer
Classifier.add(Convolution2D(32, (3, 3)))  # Add convolutional layer
Classifier.add(LeakyReLU(alpha=0.1))  # Use LeakyReLU activation
Classifier.add(BatchNormalization())
Classifier.add(MaxPooling2D(pool_size=(2, 2)))  # Add max pooling layer
Classifier.add(Dropout(0.2))

# Second Convolutional Layer
Classifier.add(Convolution2D(64, (3, 3)))  # Add another convolutional layerClassifier.add(BatchNormalization())
Classifier.add(LeakyReLU(alpha=0.1))  # Use LeakyReLU activation
Classifier.add(BatchNormalization())
Classifier.add(MaxPooling2D(pool_size=(2, 2)))  # Add another max pooling layer
Classifier.add(Dropout(0.35))

# third Convolutional Layer
Classifier.add(Convolution2D(128, (3, 3)))  # Add another convolutional layer
Classifier.add(LeakyReLU(alpha=0.1))  # Use LeakyReLU activation
Classifier.add(BatchNormalization())
Classifier.add(MaxPooling2D(pool_size=(2, 2)))  # Add another max pooling layer
Classifier.add(Dropout(0.5))
         
# Step 5 - Flattening
Classifier.add(Flatten())  # Flatten the feature maps into a 1D vector

# Step 6 - Full Connection
Classifier.add(Dense(128, activation = 'linear')) # Fully connected layer with linear activation
Classifier.add(LeakyReLU(alpha=0.1))  # Use LeakyReLU activation
Classifier.add(Dropout(0.5))
Classifier.add(Dense(5, activation = 'softmax'))  # Output layer with softmax activation (7 classes)

# Step 7 - Compiling the CNN
opt = Adam(learning_rate=0.001)  # Adam optimizer with a learning rate of 0.0001
Classifier.compile(optimizer=opt,
                   loss='categorical_crossentropy',  # Loss function for multi-class classification
                   metrics=['accuracy'])  # Track accuracy during training


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Callback for ReduceLROnPlateau
lr_reducer = ReduceLROnPlateau(monitor='val_loss',  # Metric to monitor
                               factor=0.5,          # Factor by which to reduce LR
                               patience=3,          # Patience before reducing LR
                               min_lr=1e-6,         # Minimum learning rate
                               verbose=1)           # Print when LR is reduced


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)

In [ ]:
# Save best model checkpoint
checkpoint = ModelCheckpoint("model_weights.h5.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
import time
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# === Start Training ===
print("⏳ Starting training...")
train_start_time = time.time()

history = Classifier.fit(
    train_set,
    epochs=100,
    validation_data=test_set,
    callbacks=[lr_reducer, checkpoint, early_stopping],
    verbose=0  # suppress output
)

train_end_time = time.time()
total_training_time = train_end_time - train_start_time

# === Evaluate Train Set ===
train_eval_start = time.time()
train_loss, train_accuracy = Classifier.evaluate(train_set, verbose=0)
train_predictions = Classifier.predict(train_set, verbose=0)
train_labels = train_set.labels
train_predicted_classes = np.argmax(train_predictions, axis=1)

train_precision = precision_score(train_labels, train_predicted_classes, average='weighted')
train_recall = recall_score(train_labels, train_predicted_classes, average='weighted')
train_f1 = f1_score(train_labels, train_predicted_classes, average='weighted')
train_eval_time = time.time() - train_eval_start

# === Evaluate Test Set ===
test_eval_start = time.time()
test_loss, test_accuracy = Classifier.evaluate(test_set, verbose=0)
test_predictions = Classifier.predict(test_set, verbose=0)
test_labels = test_set.labels
test_predicted_classes = np.argmax(test_predictions, axis=1)

test_precision = precision_score(test_labels, test_predicted_classes, average='weighted')
test_recall = recall_score(test_labels, test_predicted_classes, average='weighted')
test_f1 = f1_score(test_labels, test_predicted_classes, average='weighted')
test_eval_time = time.time() - test_eval_start


In [ ]:
# === Final Report ===
print("\n📊 Final Evaluation Summary")

print("\n✅ Training Set:")
print(f"Accuracy   : {train_accuracy * 100:.2f}%")
print(f"Precision  : {train_precision:.4f}")
print(f"Recall     : {train_recall:.4f}")
print(f"F1 Score   : {train_f1:.4f}")
print(f"Eval Time  : {train_eval_time:.2f} seconds")

print("\n✅ Testing Set:")
print(f"Accuracy   : {test_accuracy * 100:.2f}%")
print(f"Precision  : {test_precision:.4f}")
print(f"Recall     : {test_recall:.4f}")
print(f"F1 Score   : {test_f1:.4f}")
print(f"Eval Time  : {test_eval_time:.2f} seconds")

print(f"\n🕒 Total Training Time: {total_training_time:.2f} seconds")


In [ ]:
import time
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
# Evaluate and measure metrics for testing set
print("Evaluating the testing set...")
start_time = time.time()

test_loss, test_accuracy = Classifier.evaluate(test_set, verbose=2)
test_predictions = Classifier.predict(test_set, verbose=2)
test_labels = test_set.labels
test_predicted_classes = np.argmax(test_predictions, axis=1)

test_precision = precision_score(test_labels, test_predicted_classes, average='weighted')
test_recall = recall_score(test_labels, test_predicted_classes, average='weighted')
test_f1 = f1_score(test_labels, test_predicted_classes, average='weighted')

end_time = time.time()
test_execution_time = end_time - start_time

print("\nTesting Results:")
print(f"Accuracy: {test_accuracy * 100:.2f}%")
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")
print(f"F1 Score: {test_f1:.2f}")
print(f"Execution Time: {test_execution_time:.2f} seconds")

# Evaluate and measure metrics for training set
print("\nEvaluating the training set...")
start_time = time.time()

train_loss, train_accuracy = Classifier.evaluate(train_set, verbose=2)
train_predictions = Classifier.predict(train_set, verbose=2)
train_labels = train_set.labels
train_predicted_classes = np.argmax(train_predictions, axis=1)

train_precision = precision_score(train_labels, train_predicted_classes, average='weighted')
train_recall = recall_score(train_labels, train_predicted_classes, average='weighted')
train_f1 = f1_score(train_labels, train_predicted_classes, average='weighted')

end_time = time.time()
train_execution_time = end_time - start_time

print("\nTraining Results:")
print(f"Accuracy: {train_accuracy * 100:.2f}%")
print(f"Precision: {train_precision:.2f}")
print(f"Recall: {train_recall:.2f}")
print(f"F1 Score: {train_f1:.2f}")
print(f"Execution Time: {train_execution_time:.2f} seconds")

train_end_time = time.time()
train_time = train_end_time - train_start_time
print(f"\nTraining completed in {train_time:.2f} seconds.")

# Measure testing time
print("\nStarting testing...")
test_start_time = time.time()

test_loss, test_accuracy = Classifier.evaluate(test_set, verbose=2)

test_end_time = time.time()
test_time = test_end_time - test_start_time
print(f"\nTesting completed in {test_time:.2f} seconds.")

# Print test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# Plot Training vs Validation Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy over Epochs')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Ground truth labels from the test set
y_true = test_set.classes

# Get class names in order
class_names = ['highfive', 'handshake', 'kick left leg', 'punch right fist', 'push']

# Predict probabilities
y_pred_probs = Classifier.predict(test_set)

# Convert probabilities to predicted class indices
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(8, 6))  # Create figure and axes
disp.plot(cmap='Blues', values_format='d', ax=ax)  # Plot on specific axes
plt.title("Confusion Matrix")
plt.grid(False)
plt.show()
